In [23]:
import pandas as pd
import numpy as np
from nltk import *
import re

In [52]:
df = pd.read_csv('tmdb_movies_data.csv')
df.head()
df.count()

id                      10866
imdb_id                 10856
popularity              10866
budget                  10866
revenue                 10866
original_title          10866
cast                    10790
homepage                 2936
director                10822
tagline                  8042
keywords                 9373
overview                10862
runtime                 10866
genres                  10843
production_companies     9836
release_date            10866
vote_count              10866
vote_average            10866
release_year            10866
budget_adj              10866
revenue_adj             10866
dtype: int64

In [25]:
df.describe()
#df.count()

id                      10866
imdb_id                 10856
popularity              10866
budget                  10866
revenue                 10866
original_title          10866
cast                    10790
homepage                 2936
director                10822
tagline                  8042
keywords                 9373
overview                10862
runtime                 10866
genres                  10843
production_companies     9836
release_date            10866
vote_count              10866
vote_average            10866
release_year            10866
budget_adj              10866
revenue_adj             10866
dtype: int64

In [53]:
df = df[df['vote_average'] >6][df['vote_count']>50]
df.head()
df.count()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11048\1776468526.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['vote_average'] >6][df['vote_count']>50]


id                      2769
imdb_id                 2769
popularity              2769
budget                  2769
revenue                 2769
original_title          2769
cast                    2758
homepage                1049
director                2769
tagline                 2481
keywords                2672
overview                2769
runtime                 2769
genres                  2768
production_companies    2742
release_date            2769
vote_count              2769
vote_average            2769
release_year            2769
budget_adj              2769
revenue_adj             2769
dtype: int64

In [27]:
df.columns

Index(['id', 'imdb_id', 'popularity', 'budget', 'revenue', 'original_title',
       'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview',
       'runtime', 'genres', 'production_companies', 'release_date',
       'vote_count', 'vote_average', 'release_year', 'budget_adj',
       'revenue_adj'],
      dtype='object')

In [28]:
df = df[['popularity', 'budget', 'revenue', 'original_title',
       'cast', 'director', 'keywords', 'overview', 'genres', 'vote_average', 'imdb_id']]

In [29]:
df['net_profit'] = df['revenue'] - df['budget']
df = df.drop(['budget', 'revenue'], axis=1)

In [30]:
def verticalBar_remover(df, col_name):
    new = []
    for item in df[col_name]:
        if type(item) != str:
            item = str(item)
    # Alternative code to get str out of list # " ".join(['sahaj', 'oo'])
            
        y = ''
        for i in item.split('|'):
            y = y + ' ' + i.replace(" ", "")   #y = y + ',' + ' ' + i
                 
        new.append(y.strip())
    #print(new)
    df[col_name] = new

In [31]:
clean_df = df
for col_name in ['cast','director', 'keywords', 'genres']:
    verticalBar_remover(clean_df, col_name)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
from nltk.corpus import stopwords 

def tokeniser(df, col):
    sw = list(set(stopwords.words('english'))) + ['.', ',', '?']
    li = []
    for sentt in df[col]:
        words = word_tokenize(str(sentt).lower())
        words = [w for w in words if w not in sw]
        final = ''
        for x in words:
            final = final + " " + x
        li.append(final)
    df[col] = li

tokeniser(clean_df, 'overview')

In [34]:
def numbers_normalised(df, col_list):
    for col in col_list:
        maxx = df[col].max()
        minn = df[col].min()
        df[col] = round((df[col] - minn) / (maxx - minn), 3) 
    return df.head()

numbers_normalised(clean_df, ['popularity', 'vote_average', 'net_profit' ])

,popularity,original_title,cast,director,keywords,overview,genres,vote_average,imdb_id,net_profit
0,1.000,Jurassic World,ChrisPratt BryceDallasHoward IrrfanKhan Vincen...,ColinTrevorrow,monster dna tyrannosaurusrex velociraptor island,twenty-two years events jurassic park isla nu...,Action Adventure ScienceFiction Thriller,0.167,tt0369610,0.601
1,0.862,Mad Max: Fury Road,TomHardy CharlizeTheron HughKeays-Byrne Nichol...,GeorgeMiller,future chase post-apocalyptic dystopia australia,apocalyptic story set furthest reaches planet...,Action Adventure ScienceFiction Thriller,0.417,tt1392190,0.217
2,0.398,Insurgent,ShaileneWoodley TheoJames KateWinslet AnselElg...,RobertSchwentke,basedonnovel revolution dystopia sequel dystop...,beatrice prior must confront inner demons con...,Adventure ScienceFiction Thriller,0.083,tt2908446,0.203
3,0.339,Star Wars: The Force Awakens,HarrisonFord MarkHamill CarrieFisher AdamDrive...,J.J.Abrams,android spaceship jedi spaceopera 3d,thirty years defeating galactic empire han so...,Action Adventure ScienceFiction Fantasy,0.583,tt2488496,0.771
4,0.283,Furious 7,VinDiesel PaulWalker JasonStatham MichelleRodr...,JamesWan,carrace speed revenge suspense car,deckard shaw seeks revenge dominic toretto fa...,Action Crime Thriller,0.500,tt2820852,0.585


In [35]:
clean_df['final_numeric_weights'] = clean_df['vote_average'] + clean_df['net_profit'] + clean_df['popularity']

In [36]:
# Left out overview from dataframe
textual_df = clean_df[['original_title' , 'cast', 'director','keywords', 'genres', 'imdb_id']].apply(lambda x: x.astype(str).str.lower())
numeric_df = clean_df['final_numeric_weights']
#type(numeric_df['net_profit'][0])
final_df = textual_df.merge(numeric_df, left_index=True, right_index=True)
# final_df['text'] = final_df['cast'] + final_df['director'] + final_df['genres'] + final_df['keywords']
# final_df = final_df[['text', 'popularity', 'vote_average', 'net_profit' ]]
final_df.head()

,original_title,cast,director,keywords,genres,imdb_id,final_numeric_weights
0,jurassic world,chrispratt brycedallashoward irrfankhan vincen...,colintrevorrow,monster dna tyrannosaurusrex velociraptor island,action adventure sciencefiction thriller,tt0369610,1.768
1,mad max: fury road,tomhardy charlizetheron hughkeays-byrne nichol...,georgemiller,future chase post-apocalyptic dystopia australia,action adventure sciencefiction thriller,tt1392190,1.496
2,insurgent,shailenewoodley theojames katewinslet anselelg...,robertschwentke,basedonnovel revolution dystopia sequel dystop...,adventure sciencefiction thriller,tt2908446,0.684
3,star wars: the force awakens,harrisonford markhamill carriefisher adamdrive...,j.j.abrams,android spaceship jedi spaceopera 3d,action adventure sciencefiction fantasy,tt2488496,1.693
4,furious 7,vindiesel paulwalker jasonstatham michellerodr...,jameswan,carrace speed revenge suspense car,action crime thriller,tt2820852,1.368


In [57]:
final_df.to_csv(r'C:/Users/ASUS/Desktop/RecommendAll/final_df_movies.csv')

In [58]:
final_df.count()

original_title           2769
cast                     2769
director                 2769
keywords                 2769
genres                   2769
imdb_id                  2769
final_numeric_weights    2769
text                     2769
dtype: int64

# all in 1 count vec

In [37]:
####### All textual data in one col
test = final_df
test['text'] = final_df['cast'] + final_df['director'] + final_df['genres'] + final_df['keywords']
test = test['text']
test[:1]

0    chrispratt brycedallashoward irrfankhan vincen...
Name: text, dtype: object

In [38]:
cvv = CountVectorizer()
text_vectorised = cvv.fit_transform(test).toarray()

In [60]:
cos_sim_text = cosine_similarity(text_vectorised, text_vectorised)
np.save(r'C:/Users/ASUS/Desktop/RecommendAll/cos_sim_movies3',cos_sim_text)

In [42]:
movies = 'furious'
idx = final_df[final_df['original_title'] == movies ].index
    
text_10 = pd.Series(cos_sim_text[idx][0]).sort_values(ascending=False)[:10]
text_idx = text_10.index
text_recom_idx = (text_10 + final_df['final_numeric_weights'].loc[text_idx]).sort_values(ascending=False).index
#cast_recom = final_df.loc[cast_idx]
#print(list(final_df.loc[cast_recom_idx]))
final_df.loc[text_recom_idx]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [43]:
final_df.reindex?

# Category wise vectorised

In [44]:
#Genre Vectoriser
cv = CountVectorizer(ngram_range=(1,1))
genre_vectorised = cv.fit_transform(final_df['genres']).toarray()
# Director Vectoriser
cv2 = CountVectorizer(ngram_range=(2,3))
director_vectorised = cv.fit_transform(final_df['director']).toarray()
# Cast Vectoriser
cv3 = CountVectorizer(ngram_range=(2,3))
cast_vectorised = cv.fit_transform(final_df['cast']).toarray()

#Cosine Similarities
cos_sim_genre = cosine_similarity(genre_vectorised, genre_vectorised)
cos_sim_director = cosine_similarity(director_vectorised, director_vectorised)
cos_sim_cast = cosine_similarity(cast_vectorised, cast_vectorised)

In [45]:
print(genre_vectorised)

[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
#final_df[final_df['original_title'] == '3 idiots']
# list(final_df[final_df['director'] == 'colin trevorrow']['original_title'])

# Adding weight of Numerical data like profit etc to cosine similarity to get an overall weight involving all factors

In [46]:
idx =  [0, 1, 2]
final_df['final_numeric_weights'].loc[idx]

0    1.768
1    1.496
2    0.684
Name: final_numeric_weights, dtype: float64

In [47]:
  def recommendation(movies):
    
    idx = final_df[final_df['original_title'] == movies ].index
    
    casts = pd.Series(cos_sim_cast[idx][0]).sort_values(ascending=False)
    casts = casts[casts >= 0.49]
    cast_idx = casts.index 
    cast_recom_idx = (casts + final_df['final_numeric_weights'].loc[cast_idx]).sort_values(ascending=False).index
    cast_recom = final_df['original_title'].loc[cast_recom_idx]
    
    genres = pd.Series(cos_sim_genre[idx][0]).sort_values(ascending=False)[:10]
    genres = genres[genres >= 0.49]
    genre_idx = genres.index
    genre_recom = final_df['original_title'].loc[genre_idx]
    
    directors = pd.Series(cos_sim_director[idx][0]).sort_values(ascending=False)[:10]
    directors = directors[directors >= 0.49]
    director_idx = directors.index 
    director_recom_idx = (directors + final_df['final_numeric_weights'].loc[director_idx]).sort_values(ascending=False).index
    director_recom = final_df['original_title'].loc[director_recom_idx]
    
    print(casts)
    print(genres)
    print(directors)
    #final = pd.Series(list(cast_recom_idx) + list(director_recom_idx) + list(genre_idx)).sort_values(ascending=False).index
    #print(final_df['original_title'].loc[final][:10])
    
recommendation('the notebook')

IndexError: index 6966 is out of bounds for axis 0 with size 2769

In [50]:
x= 10
if x >= 10:
    print('h')

h


In [48]:
final_df[final_df['original_title'] == 'the notebook']

,original_title,cast,director,keywords,genres,imdb_id,final_numeric_weights,text
6966,the notebook,rachelmcadams ryangosling genarowlands jamesga...,nickcassavetes,poem river sadness fight loveofone'slife,romance drama,tt0332280,0.856,rachelmcadams ryangosling genarowlands jamesga...


In [49]:
final_df[final_df['original_title'] == 'jurassic world']

,original_title,cast,director,keywords,genres,imdb_id,final_numeric_weights,text
0,jurassic world,chrispratt brycedallashoward irrfankhan vincen...,colintrevorrow,monster dna tyrannosaurusrex velociraptor island,action adventure sciencefiction thriller,tt0369610,1.768,chrispratt brycedallashoward irrfankhan vincen...
